In [56]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

In [57]:
path='./'

In [58]:
df = pd.read_csv("../data/ccf_first_round_user_shop_behavior.csv")
shop = pd.read_csv("../data/ccf_first_round_shop_info.csv")
test = pd.read_csv("../data/evaluation_public.csv")

In [59]:
df = pd.merge(df, shop[["shop_id", "mall_id"]], how="left", on="shop_id")

In [60]:
df["time_stamp"] = pd.to_datetime(df['time_stamp'])

In [61]:
train = pd.concat([df,test])

In [62]:
mall_list = list(set(list(shop.mall_id)))

In [63]:
result = pd.DataFrame()

In [64]:
import math
def haversin(x):
    return (math.sin(x/2))**2

In [65]:
train.head()

,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos
0,32.088040,122.308291,m_1409,NaN,s_2871718,2017-08-06 21:20:00,u_376,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,32.087970,122.308162,m_1409,NaN,s_2871718,2017-08-06 21:20:00,u_376,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,40.638214,117.365255,m_4079,NaN,s_181637,2017-08-02 13:10:00,u_1041,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,31.197416,121.134451,m_6587,NaN,s_609470,2017-08-13 12:30:00,u_1158,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,31.351320,122.255867,m_3005,NaN,s_3816766,2017-08-25 19:50:00,u_1654,b_39004150|-66|false;b_39004148|-58|false;b_21...


In [66]:
len(set(mall_list))

97

In [69]:
step = 1
for mall in mall_list:
    print("目前正在跑第 %d 个mall, %d/79" % (step, step))
    train1 = train[train.mall_id == mall].reset_index(drop=True)
    train1["distance"]=0
    EARTH_RADIUS = 6371.0
    for index, row in train1.iterrows():
        ################# 剔除异常值 （距离计算）###################
        tmp_df = shop[(shop["shop_id"] == row["shop_id"]) & (shop["mall_id"] == row["mall_id"])]
        distance = 0
        if not tmp_df.empty:
            for _, row2 in tmp_df.iterrows():
                vLon = row["longitude"] - row2["longitude"]
                vLat = row["latitude"] - row2["latitude"]
                distance = haversin(vLat) + math.cos(row["latitude"])*math.cos(row2["latitude"])*haversin(vLon)
                distance = 2 * EARTH_RADIUS * math.sin(math.sqrt(distance))
                train1.loc[train1.index==index, "distance"] = distance
    train1.to_csv("../middlefile/train_v1.csv", index=False)
    step += 1

目前正在跑第 1 个mall, 1/79
目前正在跑第 2 个mall, 2/79
目前正在跑第 3 个mall, 3/79
目前正在跑第 4 个mall, 4/79
目前正在跑第 5 个mall, 5/79
目前正在跑第 6 个mall, 6/79
目前正在跑第 7 个mall, 7/79
目前正在跑第 8 个mall, 8/79
目前正在跑第 9 个mall, 9/79
目前正在跑第 10 个mall, 10/79
目前正在跑第 11 个mall, 11/79
目前正在跑第 12 个mall, 12/79
目前正在跑第 13 个mall, 13/79
目前正在跑第 14 个mall, 14/79
目前正在跑第 15 个mall, 15/79
目前正在跑第 16 个mall, 16/79
目前正在跑第 17 个mall, 17/79
目前正在跑第 18 个mall, 18/79
目前正在跑第 19 个mall, 19/79
目前正在跑第 20 个mall, 20/79
目前正在跑第 21 个mall, 21/79
目前正在跑第 22 个mall, 22/79
目前正在跑第 23 个mall, 23/79
目前正在跑第 24 个mall, 24/79
目前正在跑第 25 个mall, 25/79
目前正在跑第 26 个mall, 26/79
目前正在跑第 27 个mall, 27/79
目前正在跑第 28 个mall, 28/79
目前正在跑第 29 个mall, 29/79
目前正在跑第 30 个mall, 30/79
目前正在跑第 31 个mall, 31/79
目前正在跑第 32 个mall, 32/79
目前正在跑第 33 个mall, 33/79
目前正在跑第 34 个mall, 34/79
目前正在跑第 35 个mall, 35/79
目前正在跑第 36 个mall, 36/79
目前正在跑第 37 个mall, 37/79
目前正在跑第 38 个mall, 38/79
目前正在跑第 39 个mall, 39/79
目前正在跑第 40 个mall, 40/79
目前正在跑第 41 个mall, 41/79
目前正在跑第 42 个mall, 42/79
目前正在跑第 43 个mall, 43/79
目前正在跑第 44 个mall, 44/79
目前正在跑第

In [74]:
train_v1 = pd.read_csv("../middlefile/train_v1.csv")

In [75]:
train_v1.head()

,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos,distance
0,38.427641,122.177725,m_8379,NaN,s_3236609,2017-08-03 15:30:00,u_11838,b_49966386|-64|false;b_14938641|-42|false;b_50...,3.620469
1,38.428478,122.177856,m_8379,NaN,s_783060,2017-08-20 13:20:00,u_14015,b_21662957|-60|false;b_24930044|-59|false;b_24...,2.255627
2,38.428191,122.178484,m_8379,NaN,s_390106,2017-08-05 10:20:00,u_46728,b_50440577|-68|true;b_1114492|-62|false;b_1114...,0.627477
3,38.429275,122.176619,m_8379,NaN,s_538357,2017-08-18 21:30:00,u_120564,b_17057195|-43|false;b_21197374|-85|false;b_17...,0.627073
4,38.428335,122.178380,m_8379,NaN,s_3555397,2017-08-25 16:20:00,u_150354,b_29239883|-64|false;b_20355354|-59|false;b_32...,1.287818


In [79]:
weekday_weight = {
    0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 2, 7: 2
}
hour_weight = {
    0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1,
    6: 1, 7: 2, 8: 4, 9: 4, 10: 2, 11: 8,
    12: 8, 13: 8, 14: 4, 15: 4, 16: 4, 17: 8,
    18: 8, 19: 16, 20: 16, 21: 8, 22: 2, 23: 1
}

def get_date_and_time(train):
    train.loc[:, 'time'] = pd.to_datetime(train.time_stamp)
    train['weekday_time'] = train.time.dt.weekday + 1
    train['hour_time']  = train.time.dt.hour
    train['hour_weight'] = train.hour_time.map(hour_weight)
    train['weekday_weight'] = train.weekday_time.map(weekday_weight)
    return train.drop(['time'], axis=1)


In [83]:
step=1
for mall in mall_list:
    print("################# ######目前正在跑第 %d 个mall, %d/79##########################" % (step, step))
    train1 = train[train.mall_id == mall].reset_index(drop=True)
    train1["distance"]=0
    EARTH_RADIUS = 6371.0
    for index, row in train1.iterrows():
        ################# 剔除异常值 （距离计算）###################
        tmp_df = shop[(shop["shop_id"] == row["shop_id"]) & (shop["mall_id"] == row["mall_id"])]
        distance = 0
        if not tmp_df.empty:
            for _, row2 in tmp_df.iterrows():
                vLon = row["longitude"] - row2["longitude"]
                vLat = row["latitude"] - row2["latitude"]
                distance = haversin(vLat) + math.cos(row["latitude"])*math.cos(row2["latitude"])*haversin(vLon)
                distance = 2 * EARTH_RADIUS * math.sin(math.sqrt(distance))
                train1.loc[train1.index==index, "distance"] = distance
#     train1 = train1[train1["distance"] < 100]
    
    train1  = get_date_and_time(train1)
    
    l = []
    wifi_dict = {}
    for index, row in train1.iterrows():  
        r = {}
        wifi_list = [wifi.split('|') for wifi in row["wifi_infos"].split(";")]
        for i in wifi_list:
            r[i[0]] = int(i[1])
            if i[0] not in wifi_dict:
                wifi_dict[i[0]] = 1
            else:
                wifi_dict[i[0]] += 1
        l.append(r)
    delete_wifi = []
    for i in wifi_dict:
        ### 这里可以重大改进
        if wifi_dict[i] < 20:
            delete_wifi.append(i)
    m = []
    for row in l:
        new = {}
        for n in row.keys():
            if n not in delete_wifi:
                new[n] = row[n]
        m.append(new)
    train1 = pd.concat([train1, pd.DataFrame(m)], axis=1)
    df_train = train1[train1.shop_id.notnull()]
    df_test = train1[train1.shop_id.isnull()]
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train["shop_id"].values))
    df_train['label'] = lbl.transform(list(df_train["shop_id"].values))
    num_class = df_train['label'].max() + 1
    params = {
        'objective' : 'multi:softmax',
        'eta': 0.1,
        'max_depth' : 9,
        'eval_metric' : 'merror',
        'seed' : 0,
        'missing' : -999,
        'num_class' : num_class,
        'silent' : 1
    }
    feature = [x for x in train1.columns if x not in ['user_id', 'label', 'shop_id', 'time_stamp', 'mall_id', 'wifi_infos']]
    xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
    xgbtest = xgb.DMatrix(df_test[feature])
    watchlist = [ (xgbtrain, 'train'), (xgbtrain, 'test')]
    num_rounds = 60
    model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=15)
    df_test['label'] = model.predict(xgbtest)
    df_test['shop_id'] = df_test['label'].apply(lambda x : lbl.inverse_transform(int(x)))
    r = df_test[['row_id', 'shop_id']]
    result = pd.concat([result, r])
    result['row_id'] = result['row_id'].astype('int')
    result = result.drop_duplicates()
    result.to_csv("../result/sub.csv", index=False)
    step += 1
    print("============================ complete！ ====================================")

################# ######目前正在跑第 1 个mall, 1/79##########################


E:\anacoda\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-merror:0.076302	test-merror:0.076302
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.059189	test-merror:0.059189
[2]	train-merror:0.048869	test-merror:0.048869
[3]	train-merror:0.04168	test-merror:0.04168
[4]	train-merror:0.038857	test-merror:0.038857
[5]	train-merror:0.035902	test-merror:0.035902
[6]	train-merror:0.03352	test-merror:0.03352
[7]	train-merror:0.030521	test-merror:0.030521
[8]	train-merror:0.028536	test-merror:0.028536
[9]	train-merror:0.026816	test-merror:0.026816
[10]	train-merror:0.024964	test-merror:0.024964
[11]	train-merror:0.023773	test-merror:0.023773
[12]	train-merror:0.022185	test-merror:0.022185
[13]	train-merror:0.020818	test-merror:0.020818
[14]	train-merror:0.019406	test-merror:0.019406
[15]	train-merror:0.018348	test-merror:0.018348
[16]	train-merror:0.017642	test-merror:0.017642
[17]	train-merror:0.016539	test-merror:0.016539
[18

E:\anacoda\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\anacoda\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


============================ complete！ ====================================
################# ######目前正在跑第 2 个mall, 2/79##########################
[0]	train-merror:0.151283	test-merror:0.151283
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.112833	test-merror:0.112833
[2]	train-merror:0.095884	test-merror:0.095884
[3]	train-merror:0.080194	test-merror:0.080194
[4]	train-merror:0.073705	test-merror:0.073705
[5]	train-merror:0.064891	test-merror:0.064891
[6]	train-merror:0.058305	test-merror:0.058305
[7]	train-merror:0.054044	test-merror:0.054044
[8]	train-merror:0.04862	test-merror:0.04862
[9]	train-merror:0.043971	test-merror:0.043971
[10]	train-merror:0.040097	test-merror:0.040097
[11]	train-merror:0.037094	test-merror:0.037094
[12]	train-merror:0.03293	test-merror:0.03293
[13]	train-merror:0.031186	test-merror:0.031186
[14]	train-merror:0.027893	test-merror:0.027893


...

[25]	train-merror:0.00795	test-merror:0.00795
[26]	train-merror:0.007155	test-merror:0.007155
[27]	train-merror:0.006837	test-merror:0.006837
[28]	train-merror:0.006201	test-merror:0.006201
[29]	train-merror:0.006201	test-merror:0.006201
[30]	train-merror:0.005883	test-merror:0.005883
[31]	train-merror:0.005247	test-merror:0.005247
[32]	train-merror:0.004611	test-merror:0.004611
[33]	train-merror:0.004134	test-merror:0.004134
[34]	train-merror:0.003975	test-merror:0.003975
[35]	train-merror:0.003657	test-merror:0.003657
[36]	train-merror:0.00318	test-merror:0.00318
[37]	train-merror:0.003021	test-merror:0.003021
[38]	train-merror:0.003021	test-merror:0.003021
[39]	train-merror:0.002862	test-merror:0.002862
[40]	train-merror:0.002703	test-merror:0.002703
[41]	train-merror:0.002385	test-merror:0.002385
[42]	train-merror:0.002385	test-merror:0.002385
[43]	train-merror:0.002226	test-merror:0.002226
[44]	train-merror:0.002226	test-merror:0.002226
[45]	train-merror:0.002226	test-merror:0.002

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.045077	test-merror:0.045077
[2]	train-merror:0.037844	test-merror:0.037844
[3]	train-merror:0.033084	test-merror:0.033084
[4]	train-merror:0.029491	test-merror:0.029491
[5]	train-merror:0.026738	test-merror:0.026738
[6]	train-merror:0.023845	test-merror:0.023845
[7]	train-merror:0.022119	test-merror:0.022119
[8]	train-merror:0.020672	test-merror:0.020672
[9]	train-merror:0.019132	test-merror:0.019132
[10]	train-merror:0.017872	test-merror:0.017872
[11]	train-merror:0.016986	test-merror:0.016986
[12]	train-merror:0.016146	test-merror:0.016146
[13]	train-merror:0.015492	test-merror:0.015492
[14]	train-merror:0.014979	test-merror:0.014979
[15]	train-merror:0.014559	test-merror:0.014559
[16]	train-merror:0.014186	test-merror:0.014186
[17]	train-merror:0.013532	test-merror:0.013532
[18]	train-merror:0.013159	test-merror:0.013159
[19]	train-merror:0.012413	test-merror:0.012413
[20]	train-merror:0.011946	test-merror

[41]	train-merror:0.002318	test-merror:0.002318
[42]	train-merror:0.002196	test-merror:0.002196
[43]	train-merror:0.002074	test-merror:0.002074
[44]	train-merror:0.001952	test-merror:0.001952
[45]	train-merror:0.00183	test-merror:0.00183
[46]	train-merror:0.00183	test-merror:0.00183
[47]	train-merror:0.001708	test-merror:0.001708
[48]	train-merror:0.001586	test-merror:0.001586
[49]	train-merror:0.001586	test-merror:0.001586
[50]	train-merror:0.001464	test-merror:0.001464
[51]	train-merror:0.001464	test-merror:0.001464
[52]	train-merror:0.001342	test-merror:0.001342
[53]	train-merror:0.001098	test-merror:0.001098
[54]	train-merror:0.000976	test-merror:0.000976
[55]	train-merror:0.000976	test-merror:0.000976
[56]	train-merror:0.000976	test-merror:0.000976
[57]	train-merror:0.000976	test-merror:0.000976
[58]	train-merror:0.000976	test-merror:0.000976
[59]	train-merror:0.000854	test-merror:0.000854
============================ complete！ ====================================
################

[15]	train-merror:0.050303	test-merror:0.050303
[16]	train-merror:0.047588	test-merror:0.047588
[17]	train-merror:0.045455	test-merror:0.045455
[18]	train-merror:0.043399	test-merror:0.043399
[19]	train-merror:0.041227	test-merror:0.041227
[20]	train-merror:0.039016	test-merror:0.039016
[21]	train-merror:0.037038	test-merror:0.037038
[22]	train-merror:0.03475	test-merror:0.03475
[23]	train-merror:0.032695	test-merror:0.032695
[24]	train-merror:0.030911	test-merror:0.030911
[25]	train-merror:0.029049	test-merror:0.029049
[26]	train-merror:0.027692	test-merror:0.027692
[27]	train-merror:0.026024	test-merror:0.026024
[28]	train-merror:0.024395	test-merror:0.024395
[29]	train-merror:0.023115	test-merror:0.023115
[30]	train-merror:0.022107	test-merror:0.022107
[31]	train-merror:0.020866	test-merror:0.020866
[32]	train-merror:0.019547	test-merror:0.019547
[33]	train-merror:0.018461	test-merror:0.018461
[34]	train-merror:0.017802	test-merror:0.017802
[35]	train-merror:0.016987	test-merror:0.0

[55]	train-merror:0.00096	test-merror:0.00096
[56]	train-merror:0.0008	test-merror:0.0008
[57]	train-merror:0.0008	test-merror:0.0008
[58]	train-merror:0.0008	test-merror:0.0008
[59]	train-merror:0.0008	test-merror:0.0008
============================ complete！ ====================================
################# ######目前正在跑第 96 个mall, 96/79##########################
[0]	train-merror:0.088313	test-merror:0.088313
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.068819	test-merror:0.068819
[2]	train-merror:0.058903	test-merror:0.058903
[3]	train-merror:0.051869	test-merror:0.051869
[4]	train-merror:0.045512	test-merror:0.045512
[5]	train-merror:0.039834	test-merror:0.039834
[6]	train-merror:0.036613	test-merror:0.036613
[7]	train-merror:0.034155	test-merror:0.034155
[8]	train-merror:0.031274	test-merror:0.031274
[9]	train-merror:0.028986	test-merror:0.028986
[10]	train-m

In [29]:
subm = pd.read_csv("../result/sub.csv")

In [31]:
subm.shape

(501989, 2)

In [32]:
subm2 = pd.read_csv("../result/sub0.csv")

In [33]:
subm2.shape

(483931, 2)

In [34]:
subm.head()

,row_id,shop_id
0,118956,s_303701
1,119152,s_3040029
2,119161,s_383834
3,119201,s_873312
4,119214,s_643398


In [36]:
len(subm["row_id"].unique())

482005

In [37]:
subm = subm.drop_duplicates()

In [38]:
subm.shape

(482005, 2)

In [39]:
subm.to_csv("../result/sub.csv", index=False)

In [40]:
subm = pd.merge(subm, subm2, on=['row_id'])

,row_id,shop_id
0,118796,s_462041
1,118827,s_1966525
2,118946,s_8161
3,118980,s_1226168
4,119006,s_1083
